## DeepSeek R1 - Quantum Computing
Review of quantum computing challenges with Arxiv Retriever and Langchain orchestrator

### Environment Setup

In [1]:
# Importing required packages
import os
from azure.identity import DefaultAzureCredential
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_community.retrievers import ArxivRetriever
from langchain.prompts import PromptTemplate

In [2]:
# Defining model's endpoint and Azure credentials
DS_Model = os.getenv("AZURE_FOUNDRY_DEEPSEEK")
DS_Region = "eastus"
DS_Endpoint = f"https://{DS_Model}.{DS_Region}.models.ai.azure.com"

Az_Credential = DefaultAzureCredential()

### DeepSeek R1 Model

In [3]:
# Initiating Azure AI Foundry client
model = AzureAIChatCompletionsModel(
    endpoint = DS_Endpoint,
    credential = Az_Credential,
    max_tokens = 2048
)

### Arxiv Retriever

In [4]:
# Initiliaisng Arxiv Retriever
arxiv_retriever = ArxivRetriever(
    load_max_docs=2,
    get_ful_documents=True,
)

### Query Chain

In [5]:
# Prompt template
prompt_template = PromptTemplate(
    input_variables = ["query", "relevant_info"],
    template = """
    You are an expert scientific researcher. Use the following information from arXiv to answer the user's question. If there is no sufficient information, say 'I need more information to answer this question'.

    Question: {query}

    Relevant Information:
    {relevant_info}

    Answer:
    """
)

In [6]:
# Defining the chain
chain = prompt_template | model

### Long-response Completion

In [7]:
# Sample Arxiv query
query = "Summarise the key challenges in quantum computing research."

In [8]:
# Analysing Arxiv research papers
try:
    relevant_papers = arxiv_retriever.invoke(query)
    relevant_info = "\n".join([paper.page_content for paper in relevant_papers])
    response = chain.invoke({"query": query, "relevant_info": relevant_info})
    print("Response:", response.content)
    print("----------------------------------------")
    print("Usage:")
    print("\tPrompt tokens:", response.usage_metadata["input_tokens"])
    print("\tCompletion tokens:", response.usage_metadata["output_tokens"])
    print("\tTotal tokens:", response.usage_metadata["total_tokens"])

except Exception as e:
    print(f"Error processing Arxiv query: {e}")


Response: <think>
Okay, let's tackle this question. The user wants a summary of the key challenges in quantum computing research, based on the provided arXiv information.

First, I'll parse through the given information. The first two sections are about lay summarization and automated summarization systems in biomedical research. They mention challenges like token length limits in transformer models, memory and compute requirements for long texts, dataset limitations, and the need for hybrid methods combining extractive and abstractive approaches. But these seem to be about the summarization tasks themselves, not directly about quantum computing challenges.

Then there's the third section about quantum information technologies. It states that quantum computing has significant ethical consequences due to its unique features. The main points here are the lack of structured research into ethical implications, the need for ethical analysis considering quantum-specific features, and how qua

### Streaming Completion

In [9]:
# Streaming the chain - first 1000 words only
max_words_to_display = 1000
words_displayed = 0

try:
    relevant_papers = arxiv_retriever.invoke(query)
    relevant_info = "\n".join([paper.page_content for paper in relevant_papers])
    response = chain.stream({"query": query, "relevant_info": relevant_info})   
    for chunk in response:
        words_in_chunk = len(chunk.content.split())
        if words_displayed + words_in_chunk > max_words_to_display:
            print("\n... (Stopping stream after 1000 words)")
            break
        print(chunk.content, end="", flush=True)
        words_displayed += words_in_chunk

except Exception as e:
    print(f"Error processing Arxiv query: {e}")

<think>
Okay, the user is asking for a summary of the key challenges in quantum computing research, using the provided arXiv information. Let's start by going through the relevant information thoroughly.

First, the provided text has three main sections. The first talks about lay summarisation in biomedical articles, which seems more about making scientific papers accessible. The second discusses automated summarisation systems for research papers, mentioning challenges like token limits in models and memory requirements. The third section is about quantum information technologies, focusing on ethical implications and the lack of structured research in that area.

The user's question is about quantum computing challenges. The third section is the most relevant here. It mentions that quantum technologies (computing, communication, sensing) have significant ethical consequences but there's little structured research on their ethical implications. The paper proposes a roadmap for ethical 